In [1]:
import pandas as pd

CHESS = '/kaggle/input/top-chess-players-in-the-world/top_chess_players.csv'

# we need to drop some rows with missing values in both Title and Rating/Ranking
df = pd.read_csv(filepath_or_buffer=CHESS).dropna()
# and we need to split the ratings from the rankings and make them numeric
df['Rating'] = df['Rating | Ranking'].apply(func=lambda x: int(x.split('|')[0]))
df['Ranking']= df['Rating | Ranking'].apply(func=lambda x: int(x.split('|')[1][2:]))
df.head()

,Title,Player,Rating | Ranking,Federation,Rating,Ranking
0,GM,Magnus Carlsen,2830 | #1,Norway,2830,1
1,GM,Fabiano Caruana,2804 | #2,United States,2804,2
2,GM,Hikaru Nakamura,2788 | #3,United States,2788,3
3,GM,Ding Liren,2780 | #4,China,2780,4
4,GM,Alireza Firouzja,2759 | #5,France,2759,5


In [2]:
from plotly.express import scatter
scatter(data_frame=df, y='Rating', x='Ranking', color='Title', hover_name='Player', log_y=False)

/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


It's amazing how much a handful of players stand out at the top of the ratings/rankings; this is a very smooth curve with a little bit of noise.

In [3]:
from plotly.express import histogram
histogram(data_frame=df, x='Rating', color='Title')

/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



It is surprising that the right tail would be so thin, isn't it?

In [4]:
from plotly.express import histogram
histogram(data_frame=df, y='Federation', height=1500, color='Title')

/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [5]:
df.tail(n=10)

,Title,Player,Rating | Ranking,Federation,Rating,Ranking
1059,IM,Jubin Jimmy,2452 | #1063,India,2452,1063
1061,IM,Tigran S Petrosyan,2452 | #1063,Armenia,2452,1063
1063,IM,Benoit Lepelletier,2452 | #1063,France,2452,1063
1064,IM,Eray Kilic,2452 | #1063,Türkiye,2452,1063
1065,GM,Gudmundur Kjartansson,2452 | #1063,Iceland,2452,1063
1066,GM,Sergey Kasparov,2452 | #1063,FIDE,2452,1063
1067,IM,Xianliang Xu,2452 | #1063,Germany,2452,1063
1068,IM,Martin Lokander,2452 | #1063,Sweden,2452,1063
1069,GM,Pavel Simacek,2452 | #1063,Czech Republic,2452,1063
1070,GM,Murtas Kazhgaleyev,2452 | #1063,Kazakhstan,2452,1063
